In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Get API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is 

# Built-in Tool - DuckDuckgo Search

In [4]:
!pip install -U ddgs
from langchain_community.tools import DuckDuckGoSearchRun
search_tool=DuckDuckGoSearchRun()
results=search_tool.invoke('ipl news')
print(results)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 67.3 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0
Rules and format The IPL has several rules which vary from the established laws of cricket or those used in other Twenty20 leagues: IPL games incorporate television timeouts. Each team … The official IPL website with live scores, match updates, team rankings, all the latest news, and videos . Follow your favorite teams and players! IPL 2023: Get the latest news on T20 IPL live score, points table, schedule... Get live cricket scores, cricket updates of upcoming International, domestic and T20 matches. Catch all the latest videos, news about cricket on ESPNcricinfo. Get the latest Cricket news , live match updates, ICC schedules, photos, videos, player stats, and full coverage of International, domestic, and T20 matches on news 18.


# Built-in Tool - Shell Tool

In [7]:
!pip install langchain-experimental
from langchain_community.tools import ShellTool
shell_tool=ShellTool()
results=shell_tool.invoke('python')
print(results)

Executing command:
 python



/usr/local/lib/python3.12/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


# Custom Tools

## using decorators

In [8]:
from langchain_core.tools import tool
@tool
def multiply(a:int,b:int)->int:
  """Multiply two numbers"""
  return a*b


In [9]:
result=multiply.invoke({'a':3,'b':4})
print(result)

12


In [10]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [11]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## using structured tool

In [14]:
from langchain.tools import StructuredTool
from pydantic import BaseModel,Field

In [15]:
class MultiplyInput(BaseModel):
  a:int=Field(required=True,description="the first number to add")
  b:int=Field(required=True,description="the second number to add")

In [18]:
def multiply_func(a:int,b:int)->int:
  return a*b

In [19]:
multiply_tool=StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="multiply two numbers",
    args_schema=MultiplyInput

)

In [20]:
result=multiply_tool.invoke({'a':3,'b':4})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

12
multiply
multiply two numbers
{'a': {'description': 'the first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'the second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


# using base tool class

In [21]:
from langchain.tools import BaseTool
from typing import Type

In [22]:
class MultiplyInput(BaseModel):
  a:int=Field(required=True,description="the first number to add")
  b:int=Field(required=True,description="the second number to add")

In [23]:
class MultiplyTool(BaseTool):
  name:str="multiply"
  description:str="multiply two numbers"
  args_schema: Type[BaseModel]=MultiplyInput

  def _run(self,a:int,b:int)->int:
    return a*b


In [24]:
multipy_tool=MultiplyTool()

In [25]:
result=multiply_tool.invoke({'a':3,'b':4})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

12
multiply
multiply two numbers
{'a': {'description': 'the first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'the second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}
